In [1]:
%load_ext Cython

In [2]:
import scipy

In [4]:
%%cython

import   numpy as np
cimport  numpy
cimport  cython
from libc.math cimport round as cround

ctypedef numpy.float64_t DTYPE_t
# https://cython-docs2.readthedocs.io/en/latest/src/tutorial/numpy.html

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def distance_matrix(numpy.ndarray[DTYPE_t, ndim=2] pos1,
                    numpy.ndarray[DTYPE_t, ndim=2] pos2,
                    numpy.ndarray[DTYPE_t, ndim=1] dimensions,
                    bint pbc):
    
    # Py_ssize_t is a signed integer type provided by Python 
    # which covers the same range of values as is supported as NumPy array indices. 
    # It is the preferred type to use for loops over arrays.
    
    cdef Py_ssize_t x_max = pos1.shape[0]
    cdef Py_ssize_t y_max = pos2.shape[0]
    cdef Py_ssize_t x = 0
    cdef Py_ssize_t y = 0
    cdef Py_ssize_t i = 0
    cdef double    dx = 0
    cdef double    dy = 0
    cdef double    dz = 0
    cdef double   ddx = 0
    cdef double   ddy = 0
    cdef double   ddz = 0
    
    cdef numpy.ndarray[DTYPE_t, ndim=2] result = np.zeros([x_max, y_max], dtype=np.float64)
    
    cdef DTYPE_t value
    for x in range(x_max):
        for y in range(y_max):

            dx = pos1[x,0] - pos2[y,0]
            dy = pos1[x,1] - pos2[y,1]
            dz = pos1[x,2] - pos2[y,2]
            
            if pbc:
                ddx = dimensions[0] * cround(dx/dimensions[0])
                ddy = dimensions[1] * cround(dy/dimensions[1])
                ddz = dimensions[2] * cround(dz/dimensions[2])
                
            dx -= ddx
            dy -= ddy
            dz -= ddz

            value = (dx**2 + dy**2 + dz**2) ** 0.5
            result[x, y] = value
            
    return result

In [5]:
a = np.random.rand(1000, 3)
dim = np.array([1000.0, 1000.0, 10000.0])

In [7]:
%timeit distance_matrix(a, a, dim, False)

1.84 ms ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%timeit distance_matrix(a, a, dim, True)

8.67 ms ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit scipy.spatial.distance_matrix(a, a)

30.9 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
